In [1]:
%load_ext Cython

In [33]:
from collections import OrderedDict
import cPickle as pickle
import os
from os.path import abspath, dirname

os.sys.path.append(dirname(dirname(abspath('__file__'))))
from retro import powerspace, spherical_volume

In [29]:
%%cython --annotate
#%%cython --annotate

#from __future__ import division

cimport cython

from libc.math cimport ceil, floor, sqrt, cos, sin

import numpy as np
cimport numpy as np


@cython.embedsignature(True)
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
@cython.nonecheck(False)
def sphbin2cartbin(double r_max, double r_power,
                   int n_rbins, int n_thetabins, int n_phibins,
                   double x_bw, double y_bw, double z_bw,
                   int x_oversample, int y_oversample,
                   int z_oversample,
                   int antialias_factor=1):
    """
    Parameters
    ----------
    r_max : double
        Maximum r bin edge (note that r binning is assumed to start at 0)
        
    r_power : double
        Power used for regular power-law binning in radius
        
    n_rbins, n_thetabins, n_phibins : int with n_thetabins % 2 == 0 and n_phibins % 4 == 0
        Number or r, theta, and phi bins; note that while there are no phi
        bins expected in the retro tables, `n_phi` is used for binning
        up any spherical cells that did not catch at least one Cartesian
        bin. Note also that both n_thetabins and n_phibins must be even
        and refer to the entire range of theta and phi, even though we
        only work with the first octant. Therefore, we require each of these
        to define cells that fit exactly within the first octant:
        n_thetabins must be even, while n_phibins must be an integer multiple
        of 4.

    x_bw, y_bw, z_bw : double
        Cartesian binwidths in x, y, and z directions

    x_oversample, y_oversample, z_oversample : int >= 1
        Oversmapling factors. If oversampling is used, the returned indices
        array will have floating point values. E.g., a bin index with
        oversampling of 2 could have take values 0, 0.5, 1, ...
        Note that this increases the computational cost _and_ increases the
        memory footprint of the produced array(s).

    antialias_factor : int from 1 to 50
        The smallest binning unit in each dimension is divided again by
        this factor for more accruately computing the volume of overlap
        (and then the sub-binning for antialiasing is discarded). This
        therefore does not add to the memory footprint, but will increase
        the computational cost.

    Returns
    -------
    ind_arrays : list of M numpy.ndarrays each of shape (N, 3), dtype float32
        One array per spherical bin in the first octant.

    vol_arrays : list of M numpy.ndarrays each of shape (N,), dtype float32
        One array per spherical bin

    """
    DEF MAX_AA_FACTOR = 50
    
    DEF PI = 3.1415926535897932384626433827
    DEF TWO_PI = 6.2831853071795862319959269371
    DEF PI_BY_2 = 1.5707963267948965579989817343
    
    assert n_rbins >= 1 and n_thetabins >= 1 and n_phibins >= 1
    assert n_thetabins % 2 == 0
    assert n_phibins % 4 == 0
    
    assert x_oversample >= 1 and y_oversample >= 1 and z_oversample >= 1
    assert 1 <= antialias_factor <= MAX_AA_FACTOR
    
    cdef int n_quadrant_thetabins = int(ceil(<double> n_thetabins / 2.0))
    
    cdef double x_bw_os = x_bw / <double> x_oversample
    cdef double y_bw_os = y_bw / <double> y_oversample
    cdef double z_bw_os = z_bw / <double> z_oversample

    cdef double x_bw_os_aa = x_bw_os / <double> antialias_factor
    cdef double y_bw_os_aa = y_bw_os / <double> antialias_factor
    cdef double z_bw_os_aa = z_bw_os / <double> antialias_factor

    cdef double aa_vol = x_bw_os_aa * y_bw_os_aa * z_bw_os_aa
    cdef double overlap_vol
    
    cdef double x_halfbw_os_aa = x_bw_os_aa / 2.0
    cdef double y_halfbw_os_aa = y_bw_os_aa / 2.0
    cdef double z_halfbw_os_aa = z_bw_os_aa / 2.0

    cdef int n_xbins_oct_os = <int> ceil(r_max / x_bw_os)
    cdef int n_ybins_oct_os = <int> ceil(r_max / y_bw_os)
    cdef int n_zbins_oct_os = <int> ceil(r_max / z_bw_os)

    cdef int n_xbins_oct_os_aa = <int> ceil(r_max / x_bw_os_aa)
    cdef int n_ybins_oct_os_aa = <int> ceil(r_max / y_bw_os_aa)
    cdef int n_zbins_oct_os_aa = <int> ceil(r_max / z_bw_os_aa)

    cdef double x_idx_mirror_pt = -1.0 / x_oversample
    cdef double y_idx_mirror_pt = -1.0 / y_oversample
    cdef double z_idx_mirror_pt = -1.0 / z_oversample
    
    cdef double inv_r_power = 1.0 / r_power
    cdef double power_r_bin_scale = <double> n_rbins / r_max**inv_r_power
    cdef double costheta_bin_scale = <double> n_thetabins / 2.0
    cdef double dphi = TWO_PI / <double> n_phibins

    cdef int x_os_idx, y_os_idx, z_os_idx, xi, yi, zi
    cdef double x_idx, y_idx, z_idx
    cdef int r_bin_idx, theta_bin_idx, flat_bin_idx
    
    cdef double x0, y0, z0
    
    cdef double x_centers_sq[MAX_AA_FACTOR]
    cdef double rho_squares[MAX_AA_FACTOR][MAX_AA_FACTOR]
    
    cdef double x_center, y_center, z_center, x_center_sq, y_center_sq,  z_center_sq
    cdef double rho_sq

    cdef double r
    cdef double new_vol
    
    cdef double[:] phi_bin_centers = np.linspace(start=TWO_PI / <double> n_phibins / 2.0,
                                                 stop=PI_BY_2 - TWO_PI / <double> n_phibins / 2.0,
                                                 num=n_phibins)
    
    # Note: there is no phi dependence in the source tables
    
    bin_mapping = []
    for _ in range(n_rbins):
        tmp = []
        for _ in range(n_quadrant_thetabins):
            tmp.append({})
        bin_mapping.append(tmp)
        
    for x_os_idx in range(n_xbins_oct_os):
        x_idx = <double> x_os_idx / <double> x_oversample
        x0 = x_os_idx * x_bw_os + x_halfbw_os_aa
        for xi in range(antialias_factor):
            x_center = x0 + xi * x_bw_os_aa
            x_center_sq = x_center * x_center
            x_centers_sq[xi] = x_center_sq

        for y_os_idx in range(n_ybins_oct_os):
            y_idx = <double> y_os_idx / <double> y_oversample
            y0 = y_os_idx * y_bw_os + y_halfbw_os_aa
            for yi in range(antialias_factor):
                y_center = y0 + yi * y_bw_os_aa
                y_center_sq = y_center * y_center
                for xi in range(antialias_factor):
                    rho_squares[xi][yi] = x_centers_sq[xi] + y_center_sq

            for z_os_idx in range(n_zbins_oct_os):
                z_idx = <double> z_os_idx / <double> z_oversample
                
                # First quadrant
                xyz_idx_q1 = (x_idx, y_idx, z_idx)
                ## Second quadrant: y -> -x and x -> y
                #xyz_idx_q2 = (ny_idx, x_idx, z_idx)
                ## Third quadrant: x -> -x and y -> -y
                #xyz_idx_q3 = (nx_idx, ny_idx, z_idx)
                ## Fourth quadrant: x -> -y and y -> x
                #xyz_idx_q4 = (y_idx, nx_idx, z_idx)
                
                ## Same as above but mirrored about xy plane
                #xyz_idx_negz_q1 = (x_idx, y_idx, nz_idx)
                #xyz_idx_negz_q2 = (ny_idx, x_idx, nz_idx)
                #xyz_idx_negz_q3 = (nx_idx, ny_idx, nz_idx)
                #xyz_idx_negz_q4 = (y_idx, nx_idx, nz_idx)

                z0 = z_os_idx * z_bw_os + z_halfbw_os_aa
                for zi in range(antialias_factor):
                    z_center = z0 + zi * z_bw_os_aa
                    z_center_sq = z_center * z_center
                    for xi in range(antialias_factor):
                        for yi in range(antialias_factor):
                            r = sqrt(rho_squares[xi][yi] + z_center_sq)
                            if r < 0 or r >= r_max:
                                continue
                                
                            r_bin_idx = int(floor(r**inv_r_power * power_r_bin_scale))
                            theta_bin_idx = int((1.0 - z_center / r) * costheta_bin_scale)
                            if theta_bin_idx < 0 or theta_bin_idx >= n_thetabins:
                                continue
                                
                            d = bin_mapping[r_bin_idx][theta_bin_idx]
                            new_vol = aa_vol + d.get(xyz_idx_q1, 0.0)
                            d[xyz_idx_q1] = new_vol

    ind_arrays = []
    vol_arrays = []
    
    cdef double r_bmin, r_bmax, r_bcenter
    cdef double costheta_bmin, costheta_bmax, costheta_bcenter
    cdef double rho_center, phi_bin_center
    
    cdef double total_tabulated_vol, sph_bin_vol
    cdef double dr, dcostheta
    
    # Account for any spherical bins that were assigned no corresponding
    # Cartesian bins; otherwise, normalize total binned volume to be no larger
    # than that of each spherical volume element (dr and dtheta come
    # from the polar bin, while dphi is pi/2 since we only computed one
    # octant)
    for r_bin_idx in range(n_rbins):
        for theta_bin_idx in range(n_quadrant_thetabins):
            d = bin_mapping[r_bin_idx][theta_bin_idx]
            r_bmin = (<double> r_bin_idx / power_r_bin_scale)**r_power
            r_bmax = (<double> (r_bin_idx + 1) / power_r_bin_scale)**r_power
            dr = r_bmax - r_bmin

            costheta_bmin = 1 - theta_bin_idx / costheta_bin_scale
            costheta_bmax = 1 - (theta_bin_idx + 1) / costheta_bin_scale
            dcostheta = costheta_bmax - costheta_bmin
            
            # For spherical volume elements that don't have any assigned Cartesian bins,
            # find spherical element's center and locate it in a Cartesian bin
            if len(d) == 0:
                r_bcenter = (r_bmin + r_bmax) / 2.0
                costheta_bcenter = (costheta_bmin + costheta_bmax) / 2.0
                z_center = r_bcenter * costheta_bcenter
                z_idx = floor(z_center / z_bw_os) / <double> z_oversample
                rho_center = sqrt(r_bcenter**2 - z_center**2)
                sph_bin_vol = -dcostheta * (r_bmax**3 - r_bmin**3) / 3.0 * dphi

                for phi_idx in range(n_phibins):
                    phi_bin_center = phi_bin_centers[phi_idx]
                    x_center = rho_center * cos(phi_bin_center)
                    y_center = rho_center * sin(phi_bin_center)
                    
                    x_idx = floor(x_center / x_bw_os) / <double> x_oversample
                    y_idx = floor(y_center / y_bw_os) / <double> y_oversample
                    
                    # NOTE: duplicates are overwritten (i.e., should be at
                    # most one entry for xyz_idx_q1 per spherical bin)
                    d[xyz_idx_q1] = sph_bin_vol
            
            # Normalize volume weight assigned from this spherical element
            # to all associated Cartesian elements to be exactly the volume of the
            # spherical volume element (i.e., the area of the polar element rotated
            # through the first octant). We can assume this normalization to be correct
            # since we define the Cartesian binning to completely encapsulate the entire spherical
            # volume (so there won't be any "unbinned" parts of the spherical vol)
            sph_bin_vol = -dcostheta * (r_bmax**3 - r_bmin**3) / 3.0 * PI_BY_2
            vols = np.array(d.values())
            total_tabulated_vol = np.sum(vols)
            norm_factor = sph_bin_vol / total_tabulated_vol
            vols *= norm_factor
                    
            ind_arrays.append(np.array(d.keys(), dtype=np.float32))
            vol_arrays.append(vols.astype(np.float32))

    return ind_arrays, vol_arrays

In [47]:
r_max = 400
r_power = 2
n_rbins = 200
n_thetabins = 40
n_phibins = 80

r_edges = powerspace(0, r_max, n_rbins + 1, r_power)
theta_edges = np.arccos(np.linspace(1, -1, n_thetabins + 1))

R, THETA = np.meshgrid(r_edges, theta_edges, indexing='ij')
coords = []
exact_vols = []
for ri in range(n_rbins):
    subcoords = []
    sub_exact_vols = []
    for ti in range(int(np.ceil(n_thetabins / 2.0))):
        rs = R[ri:ri+2, ti:ti+2]
        ts = THETA[ri:ri+2, ti:ti+2]
        bin_corner_coords = zip(rs.flat, ts.flat)
        dcostheta = np.abs(np.diff(np.cos([ts.max(), ts.min()])))
        exact_vol = spherical_volume(rmin=rs.max(), rmax=rs.min(), dcostheta=dcostheta, dphi=np.pi/2)
        sub_exact_vols.append(exact_vol)
    exact_vols.append(sub_exact_vols)
exact_vols = np.array(exact_vols)

In [ ]:
%%time

kwargs = dict(
    r_max=r_max, r_power=r_power,
    n_rbins=n_rbins, n_thetabins=n_thetabins, n_phibins=n_phibins,
    x_bw=1, y_bw=1, z_bw=1,
    x_oversample=2, y_oversample=2, z_oversample=2,
    antialias_factor=1
)
ind_arrays, vol_arrays = sphbin2cartbin(**kwargs)

In [ ]:
%%time

dump_data = OrderedDict([('kwargs', kwargs), ('ind_arrays', ind_arrays), ('vol_arrays', vol_arrays)])
fname = (
    'sphbin2cartbin_first_octant_mapping'
    '_nr{n_rbins:d}_ntheta{n_thetabins:d}_nphi{n_phibins:d}'
    '_rmax{r_max:f}_rpwr{r_power}'
    '_xbw{x_bw:f}_ybw{y_bw:f}_zbw{z_bw:f}'
    '_xos{x_oversample:d}_yos{y_oversample:d}_zos{z_oversample:d}'
    '_aa{antialias_factor:d}'
    '.pkl'.format(**kwargs)
)
pickle.dump(dump_data, file(fname, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

!ls -hl {fname}

In [ ]:
binned_vol = np.sum([va.sum() for va in vol_arrays])
exact_vol = spherical_volume(rmin=0, rmax=r_max, dcostheta=-1, dphi=np.pi/2)
print 'exact vol = %f, binned vol = %f (%e fract error)' % (exact_vol, binned_vol, (binned_vol-exact_vol)/exact_vol)

In [ ]:
ind_bin_vols = np.array([va.sum() for va in vol_arrays])
fract_err = ind_bin_vols/exact_vols.flat - 1
abs_fract_err = np.abs(fract_err)
worst_abs_fract_err = np.max(abs_fract_err)
flat_idx = np.where(abs_fract_err == worst_abs_fract_err)[0][0]
r_idx, theta_idx = divmod(flat_idx, int(np.ceil(n_thetabins/2)))
print ('worst single-bin fract err: %e; r_idx=%d, theta_idx=%d; binned vol=%e, exact vol=%e'
       % (worst_abs_fract_err, r_idx, theta_idx, ind_bin_vols[flat_idx], exact_vols[r_idx, theta_idx]))